In [1]:
# !pip install python-mnist
#Imports and confusion matrix function
import matplotlib.pyplot as plt
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import scipy.io
import numpy as np
# from mnist import MNIST
from keras.datasets import mnist
import keras
from computerized_digits import make_training_array

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=True,
                          title=None,
                          cmap=plt.cm.Greens, save_to_file = True):
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    fig, ax = plt.subplots(figsize = (16,16))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
   # if save_to_file:
    #    plt.savefig('Assets/files/' + title + '.pdf')
    return ax

In [3]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
train_arr = make_training_array(7500)
artif_train_X = []
artif_train_Y = []
for k in train_arr.keys():
    for sample in train_arr[k]:
        artif_train_X.append(np.array(sample))
        artif_train_Y.append(int(k))
artif_train_X = np.array(artif_train_X) 
artif_train_Y = np.array(artif_train_Y)
train_X = np.vstack([artif_train_X,train_X])
train_y = np.hstack([artif_train_Y,train_y])
# Shuffle in unison
p = np.random.permutation(len(train_y))
train_X = train_X[p]
train_y = train_y[p]


In [4]:
#Create model and fit
model_mnist = RandomForestClassifier()
model_mnist.fit(train_X.reshape(train_X.shape[0],28*28),train_y)
#Model Score
prediction_mnist = model_mnist.score(test_X.reshape(test_X.shape[0], 28*28), test_y)
print(prediction_mnist)

0.9685


In [5]:
model = keras.models.Sequential([
    keras.layers.Conv2D(activation='relu', padding="SAME", filters=64, kernel_size=7, input_shape=[28, 28, 1]),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(kernel_size=3, activation='relu', padding="SAME", filters=128),
    keras.layers.Conv2D(kernel_size=3, activation='relu', padding="SAME", filters=128),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(kernel_size=3, activation='relu', padding="SAME", filters=256),
    keras.layers.Conv2D(kernel_size=3, activation='relu', padding="SAME", filters=256),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.BatchNormalization(),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(units=96, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(units=10, activation='softmax'),
])

In [6]:
# (train_X, train_y), (test_X, test_y)
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(train_X.reshape((-1,28,28,1)), train_y, epochs=10)
score = model.evaluate(test_X, test_y)
X_new = test_X[:10] # pretend we have new images
y_pred = model.predict(X_new)

Epoch 1/10
3985/3985 [==============================] - 105s 25ms/step - loss: 0.1830 - accuracy: 0.9469
Epoch 2/10
3985/3985 [==============================] - 100s 25ms/step - loss: 0.0532 - accuracy: 0.9890
Epoch 3/10
3985/3985 [==============================] - 100s 25ms/step - loss: 0.0433 - accuracy: 0.9912
Epoch 4/10
3985/3985 [==============================] - 101s 25ms/step - loss: 0.0355 - accuracy: 0.9929
Epoch 5/10
3985/3985 [==============================] - 101s 25ms/step - loss: 0.0280 - accuracy: 0.9941
Epoch 6/10
3985/3985 [==============================] - 101s 25ms/step - loss: 0.0246 - accuracy: 0.9948
Epoch 7/10
3985/3985 [==============================] - 101s 25ms/step - loss: 0.0234 - accuracy: 0.9950
Epoch 8/10
3985/3985 [==============================] - 101s 25ms/step - loss: 0.0214 - accuracy: 0.9958
Epoch 9/10
3985/3985 [==============================] - 98s 25ms/step - loss: 0.0193 - accuracy: 0.9961
Epoch 10/10
313/313 [==============================] - 2

In [7]:
list(map(lambda x: list(x).index(max(x)), y_pred)), list(test_y[:10])

([7, 2, 1, 0, 4, 1, 4, 9, 5, 9], [7, 2, 1, 0, 4, 1, 4, 9, 5, 9])

In [8]:
model.save('CNN_MNIST_COMP_v3')

INFO:tensorflow:Assets written to: CNN_MNIST_COMP_v3\assets


In [12]:
print(f'Improvement over random forest is {100*((1-prediction_mnist)/(1-score[1])):.03f}%')

Improvement over random forest is 403.847%
